In [22]:
import os
from pathlib import Path


import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
path = Path('data/titanic')
df = pd.read_csv(path/'train.csv')
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


This is an experiment in trying to use sklearn's pipelines.  Seem kind of clunky to me

In [73]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
 

# Data Preprocessing
from sklearn.model_selection import train_test_split
# reread the data
df = pd.read_csv(path/'train.csv')

# Simple imputer is a pain so I am not going to use it. 
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

modes = train_df.mode().iloc[0]  # use only training data to fill missing values
train_df.fillna(modes, inplace=True)
valid_df.fillna(modes, inplace=True)


# transform the columns , age standardize, fare take log and then standardiz
log_transformer = Pipeline(steps=[("log",FunctionTransformer(np.log1p)), ("scale",StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age']),
        ('log', log_transformer, ['Fare']),
        ('cat', OneHotEncoder(), ['Sex', 'Embarked', 'Pclass']),
        ('pass', 'passthrough', ['SibSp', 'Parch'])
    ],
    remainder='drop'
)

X = train_df.drop(columns='Survived')
y = train_df['Survived']
X_valid = valid_df.drop(columns='Survived')
y_valid = valid_df['Survived']
# Apply the preprocessing pipeline
X = preprocessor.fit_transform(X)
X_valid = preprocessor.transform(X_valid)


In [94]:
valid_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
709,710,1,3,"Moubarek, Master. Halim Gonios (""William George"")",male,24.0,1,1,2661,15.2458,C23 C25 C27,C
439,440,0,2,"Kvillner, Mr. Johan Henrik Johannesson",male,31.0,0,0,C.A. 18723,10.5000,C23 C25 C27,S
840,841,0,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,C23 C25 C27,S
720,721,1,2,"Harper, Miss. Annie Jessie ""Nina""",female,6.0,0,1,248727,33.0000,C23 C25 C27,S
39,40,1,3,"Nicola-Yarred, Miss. Jamila",female,14.0,1,0,2651,11.2417,C23 C25 C27,C
...,...,...,...,...,...,...,...,...,...,...,...,...
433,434,0,3,"Kallio, Mr. Nikolai Erland",male,17.0,0,0,STON/O 2. 3101274,7.1250,C23 C25 C27,S
773,774,0,3,"Elias, Mr. Dibo",male,24.0,0,0,2674,7.2250,C23 C25 C27,C
25,26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...",female,38.0,1,5,347077,31.3875,C23 C25 C27,S
84,85,1,2,"Ilett, Miss. Bertha",female,17.0,0,0,SO/C 14885,10.5000,C23 C25 C27,S


In [74]:
# get feature names  (yeah this is hacky af)
cat_features = preprocessor.transformers_[2][1].get_feature_names_out()
num_features = ['Age', 'LogFare']
feature_names = np.concatenate([num_features, cat_features, ['SibSp', 'Parch']])
feature_names
 

array(['Age', 'LogFare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'SibSp',
       'Parch'], dtype=object)

In [77]:
# Simple neural network model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class TitanicModel_1(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(TitanicModel_1, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

Try to reproduce results in lesson 5

In [86]:
test1 = TitanicModel_1(X.shape[1], 10)

# Convert the data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1)
X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1)


# Train the model on all rows (no batches)
 
loss_fn = nn.BCELoss()
optimizer = optim.Adam(test1.parameters(), lr=0.01)
for epoch in range(1000):
    optimizer.zero_grad()
    y_pred = test1(X_tensor)
    loss = loss_fn(y_pred, y_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch} loss: {loss.item()}')

Epoch 0 loss: 0.7364271283149719
Epoch 100 loss: 0.39753732085227966
Epoch 200 loss: 0.37152236700057983
Epoch 300 loss: 0.35802993178367615
Epoch 400 loss: 0.34854868054389954
Epoch 500 loss: 0.34291356801986694
Epoch 600 loss: 0.33810561895370483
Epoch 700 loss: 0.33239689469337463
Epoch 800 loss: 0.32976120710372925
Epoch 900 loss: 0.32851642370224


In [87]:
# Evaluate the model
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).float()
    correct = (y_pred == y_true).float().sum()
    return correct / len(y_true)
accuracy(test1(X_valid_tensor), y_valid_tensor) 

tensor(0.8380)

About the same as in lesson 5.  Ok, so now lets add in embedding layers.  

In [186]:
# Model with embedding for categorical features  
class TitanicModel_2(nn.Module):
    def __init__(self, num_features,cat_sizes, emb_size=5, hidden_size=10):
        super(TitanicModel_2, self).__init__()
        num_cat_features = len(cat_sizes)
        self.embeddings = nn.ModuleList([nn.Embedding(s, emb_size) for s in cat_sizes])
        self.fc1 = nn.Linear(num_features + num_cat_features * emb_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    # note taht we have to pass the categorical and numerical features separately    
    def forward(self, x_num, x_cat):
        x = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat(x + [x_num], dim=1)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

I need to preprocess slightly differently 

In [187]:
from sklearn.preprocessing import OrdinalEncoder

preprocessor_num = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age']),
        ('log', log_transformer, ['Fare']),
        ('pass', 'passthrough', ['SibSp', 'Parch'])
    ],
    remainder='drop'
)

processor_cat = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(), ['Sex', 'Embarked', 'Pclass'])],
        remainder='drop'
    )


X = train_df.drop(columns='Survived')
y = train_df['Survived']
X_valid = valid_df.drop(columns='Survived')
y_valid = valid_df['Survived']

# Apply the preprocessing pipeline
X_num, X_cat = preprocessor_num.fit_transform(X), processor_cat.fit_transform(X)
X_valid_num, X_valid_cat = preprocessor_num.transform(X_valid), processor_cat.transform(X_valid)
 

In [200]:
cat_df = pd.DataFrame(X_cat)
cat_sizes = [len(cat_df[c].unique()) for c in cat_df.columns]

# Convert the data to tensors
X_cat_tensor = torch.tensor(X_cat, dtype=torch.long)
X_num_tensor = torch.tensor(X_num, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).reshape(-1, 1)
X_valid_cat_tensor = torch.tensor(X_valid_cat, dtype=torch.long)
X_valid_num_tensor = torch.tensor(X_valid_num, dtype=torch.float32)
y_valid_tensor = torch.tensor(y_valid.values, dtype=torch.float32).reshape(-1, 1)

In [218]:
test2 = TitanicModel_2(X_num.shape[1], cat_sizes, 64,128)

# Train the model on all rows (no batches)
 
loss_fn = nn.BCELoss()
optimizer = optim.Adam(test2.parameters(), lr=0.001)
best_loss = loss_fn(test2(X_num_tensor, X_cat_tensor), y_tensor).item()
for epoch in range(20001):
    optimizer.zero_grad()
    y_pred = test2(X_num_tensor, X_cat_tensor)
    loss = loss_fn(y_pred, y_tensor)
    loss.backward()
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model = test2.state_dict()
    else:
        if loss.item() > 1.5 * best_loss:
            print(f'Early stopping at epoch {epoch}')
            print(f'Best loss: {best_loss}')
            print(f'Loading best model')
            test2.load_state_dict(best_model)
            break

    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch} loss: {loss.item()}')

Epoch 0 loss: 0.6499691009521484
Epoch 1000 loss: 0.298004150390625
Epoch 2000 loss: 0.2521078884601593
Epoch 3000 loss: 0.23028653860092163
Epoch 4000 loss: 0.21590085327625275
Epoch 5000 loss: 0.20686742663383484
Epoch 6000 loss: 0.20058196783065796
Epoch 7000 loss: 0.19388672709465027
Epoch 8000 loss: 0.19034336507320404
Epoch 9000 loss: 0.18693891167640686
Epoch 10000 loss: 0.1843898594379425
Epoch 11000 loss: 0.18046998977661133
Epoch 12000 loss: 0.17869967222213745
Epoch 13000 loss: 0.17465640604496002
Epoch 14000 loss: 0.17249153554439545
Epoch 15000 loss: 0.17061905562877655
Epoch 16000 loss: 0.16843271255493164
Epoch 17000 loss: 0.16643857955932617
Epoch 18000 loss: 0.16729876399040222
Epoch 19000 loss: 0.16431736946105957
Epoch 20000 loss: 0.1619330793619156


In [219]:
accuracy(test2(X_valid_num_tensor, X_valid_cat_tensor), y_valid_tensor)

tensor(0.8212)

Not any better and a lot more parameters